In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
from typing import List

import torch
from torch import nn, Tensor
import torch.nn.functional as F

import attention
import net

In [ ]:
B, T, E, D = 4, 5, 2, 10

x = torch.rand((B*T*E*D)).reshape((B, T, E*D))
print(x.shape)
print(x.mT.shape)
x = x.view((x.size(0), E, D))
x.shape

In [ ]:
B, H, W, E, D = 4, 3, 3, 2, 12

x = torch.rand((B*H*W*E*D)).reshape((B, H, W, E*D))
print(x.shape, x.shape[0], x.shape[-1])

x = x.view((*x.shape[:-1], E, x.size(-1)//E))
print(x.shape)

x = x @ x.mT
print(x.shape)

In [88]:
B, H, W, E, D = 4, 7, 7, 2, 24
i = torch.rand((B*H*W*D)).reshape((B, H, W, D))

mha = attention.MultiHeadAttention(D, E, 0.1, 0.1)
y = mha(i.reshape((B, H*W, D)))
print(y.shape)


torch.Size([4, 49, 24])


In [160]:
B, H, W, E, D = 4, 10, 10, 2, 24
mha = attention.ShiftWindowMHA([8,8], [4, 4], D, E, 0.1, 0.1)
i = torch.rand((B*H*W*D)).reshape((B, H, W, D))
mha(i).shape

torch.Size([4, 10, 10, 24])

In [132]:
mha = net.SwinAttention(
    D, [8,8], [4, 4], E, 0.1, 0.1
)
mha(i).shape

abc torch.Size([16, 2, 64, 64])
torch.Size([4, 4, 2, 64, 64]) torch.Size([4, 64, 64])


torch.Size([4, 10, 10, 24])

In [124]:
window_size = torch.tensor([4, 4])
shift_size = torch.tensor([2, 2])

size = torch.tensor([i.size(1), i.size(2)], dtype=torch.int, device=i.device)
shift = torch.where(window_size >= size, 0, shift_size)
pad = (window_size - size%window_size)%window_size
pad_img = F.pad(i, (0, 0, 0, pad[1], 0, pad[0]))

num_windows = (pad_img.size(1) // window_size[0]) * (pad_img.size(2) // window_size[1])
attn_mask = x.new_zeros([pad_img.size(1), pad_img.size(2)])
h_slices = ((0, -window_size[0]), (-window_size[0], -shift[0]), (-shift[0], None))
w_slices = ((0, -window_size[1]), (-window_size[1], -shift[1]), (-shift[1], None))
count = 0
for h in h_slices:
    for w in w_slices:
        # print(h, w)
        attn_mask[h[0] : h[1], w[0] : w[1]] = count
        count += 1
print('here', attn_mask.shape)
attn_mask = attn_mask.view(pad_img.size(1) // window_size[0], window_size[0], pad_img.size(2) // window_size[1], window_size[1])
print(attn_mask.shape, attn_mask.permute(0, 2, 1, 3).shape)
attn_mask = attn_mask.permute(0, 2, 1, 3).reshape(num_windows.prod(), window_size[0] * window_size[1])
print(attn_mask.shape)
print(attn_mask.unsqueeze(1).shape, attn_mask.unsqueeze(2).shape)
attn_mask = attn_mask.unsqueeze(1) != attn_mask.unsqueeze(2)
print(attn_mask.shape)
# for j in range(16):
# print(attn_mask[0, 0])
# print(attn_mask[1, 0])
# print(attn_mask[2, 0])
# print(attn_mask[3, 0])

here torch.Size([8, 8])
torch.Size([2, 4, 2, 4]) torch.Size([2, 2, 4, 4])
torch.Size([4, 16])
torch.Size([4, 1, 16]) torch.Size([4, 16, 1])
torch.Size([4, 16, 16])
